In [1]:
import tensorflow as tf
import pickle
import numpy as np
import math as math
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
# def create_model(channel, activation='relu', dropout=0.0, optimizer='adam', neurons=64): 
#     model = tf.keras.Sequential()
#     model.add(tf.keras.layers.Conv2D(neurons, kernel_size=3, padding='same', activation=activation, input_shape=(32, 32, 3)))
#     model.add(tf.keras.layers.MaxPool2D(pool_size=2))
#     model.add(tf.keras.layers.Dropout(dropout))

#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Dense(256, activation=activation))
#     model.add(tf.keras.layers.Dense(256, activation=activation))
#     model.add(tf.keras.layers.Dense(43, activation='softmax'))
  
#     model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
#                 optimizer=tf.keras.optimizers.Adam(), 
#                 metrics=['accuracy'])

#     model.summary()

#     return model

In [11]:
def create_model_8(channel, activation="relu", dropout=0.0, optimizer='adam', neurons=64, image_shape=(32, 32, 3)):

    # initialize the VGG16 model from the keras library
    vgg16_model = tf.keras.applications.VGG16(include_top=False,
                                              weights='imagenet',
                                              input_shape=image_shape,
                                              classes=43)

    model = tf.keras.Sequential()

    # Freeze the layers
    for layer in model.layers:
        layer.trainable = False
    
    model.add(vgg16_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.4))
    
    print(model.layers[-1].output_shape)
    
    
    model.add(tf.keras.layers.Conv2D(neurons, kernel_size=3, padding='same', activation=activation))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation=activation))
    model.add(tf.keras.layers.Dense(256, activation=activation))
    
    
    # Add 'softmax' instead of earlier 'prediction' layer.
    model.add(tf.keras.layers.Dense(43, activation='softmax'))

    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])

    model.summary()

    return model

In [4]:
def generate_plots(directory, file, model_history, epochs):
    """
    A method that takes the model history of a trained model and plots its:
    1. Training accuracy
    2. Training loss
    3. Validation accuracy
    4. Validation loss
    """
    acc = model_history.history['accuracy']
    val_acc = model_history.history['val_accuracy']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']


    plt.figure(1)
    plt.suptitle('Accuracy learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('accuracy', fontsize=14)
    plt.plot(acc, label='training accuracy')
    plt.plot(val_acc, label='validation accuracy')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="lower right")
    plt.savefig("{}/accuracy_{}.png".format(directory, file), dpi=300)

    plt.figure(2)
    plt.suptitle('Loss learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('loss', fontsize=14)
    plt.plot(loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="upper right")
    plt.savefig("{}/loss_{}.png".format(directory, file), dpi=300)

In [5]:
# Variables
EPOCHS = 50
BATCH_SIZE = 512
file = 0

In [6]:
if file==0 or file==1 or file==2 or file==3:
    channel = 3
else:
    channel = 1

# Opening file for reading in binary mode
with open('../data/data{}.pickle'.format(file), 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type

In [7]:
y_train = data['y_train']
y_val = data['y_validation']
y_test = data['y_test']

In [8]:
# Making channels come at the end
X_train = data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
X_val = data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
X_test = data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

In [9]:
# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else:
        print(i + ':', j.shape)

x_test: (12630, 32, 32, 3)
y_validation: (4410,)
x_validation: (4410, 32, 32, 3)
labels: 43
x_train: (86989, 32, 32, 3)
y_test: (12630,)
y_train: (86989,)


In [12]:
# Initialize the directories
save_model = "saved_models"
plot_dir = "plots"
results_dir = "results"

root_dir = os.path.abspath('./')
model_save_dir = os.path.join(root_dir, save_model)
plot_save_dir = os.path.join(root_dir, plot_dir)
results_save_dir = os.path.join(root_dir, results_dir)


os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(plot_save_dir, exist_ok=True)
os.makedirs(results_save_dir, exist_ok=True)


model = create_model_8(channel=channel, activation="sigmoid", dropout=0.5, optimizer="adam", neurons=128)
# model = create_model2(activation="relu")

# For enabling early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
mc = tf.keras.callbacks.ModelCheckpoint('{}/best_model_data{}.h5'.format(model_save_dir, file), monitor='val_loss', save_best_only=True, verbose=1)

(None, 512)


ValueError: Input 0 of layer conv2d_1 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 512]

In [ ]:
with tf.device('/device:GPU:0'):

    # Random Rotations
    # define data preparation
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20, brightness_range=[0.3, 0.6])
    datagen.fit(X_train)
    
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                                      validation_data=(X_val, y_val),
                                      steps_per_epoch=len(X_train) // BATCH_SIZE,
                                      epochs=EPOCHS,
                                      callbacks=[es, mc])
            

In [ ]:
predictions = model.predict(X_test, batch_size=512)

y_pred = [np.argmax(prediction) for prediction in predictions]

acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred, average="macro")
rec = recall_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred, average="macro")

In [ ]:
f = open("{}/results_{}.txt".format(results_save_dir, file), "w+")
f.write("Accuracy: {}\n".format(acc))
f.write("Precision: {}\n".format(pre))
f.write("Recall: {}\n".format(rec))
f.write("F1 score: {}\n".format(f1))
f.close()

In [ ]:
generate_plots(plot_save_dir, file, history, EPOCHS)

In [ ]:
model.save('{}/best_model_data{}.h5'.format(model_save_dir, file)) 

# Recreate the exact same model, including its weights and the optimizer
# new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
# new_model.summary()